# December 19, 2015
https://adventofcode.com/2015/day/19

In [1]:
fn = "../data/2015/19.txt"

ops = {}
with open(fn, "r") as file:
    while True:
        line = file.readline()
        if line == "\n":
            break

        start, stop = line.strip("\n").split(" => ")
        if start in ops.keys():
            ops[start].append(stop)
        else:
            ops[start] = [stop]

    compound = file.readline().strip()

In [2]:
print(compound)
ops

CRnSiRnCaPTiMgYCaPTiRnFArSiThFArCaSiThSiThPBCaCaSiRnSiRnTiTiMgArPBCaPMgYPTiRnFArFArCaSiRnBPMgArPRnCaPTiRnFArCaSiThCaCaFArPBCaCaPTiTiRnFArCaSiRnSiAlYSiThRnFArArCaSiRnBFArCaCaSiRnSiThCaCaCaFYCaPTiBCaSiThCaSiThPMgArSiRnCaPBFYCaCaFArCaCaCaCaSiThCaSiRnPRnFArPBSiThPRnFArSiRnMgArCaFYFArCaSiRnSiAlArTiTiTiTiTiTiTiRnPMgArPTiTiTiBSiRnSiAlArTiTiRnPMgArCaFYBPBPTiRnSiRnMgArSiThCaFArCaSiThFArPRnFArCaSiRnTiBSiThSiRnSiAlYCaFArPRnFArSiThCaFArCaCaSiThCaCaCaSiRnPRnCaFArFYPMgArCaPBCaPBSiRnFYPBCaFArCaSiAl


{'Al': ['ThF', 'ThRnFAr'],
 'B': ['BCa', 'TiB', 'TiRnFAr'],
 'Ca': ['CaCa', 'PB', 'PRnFAr', 'SiRnFYFAr', 'SiRnMgAr', 'SiTh'],
 'F': ['CaF', 'PMg', 'SiAl'],
 'H': ['CRnAlAr',
  'CRnFYFYFAr',
  'CRnFYMgAr',
  'CRnMgYFAr',
  'HCa',
  'NRnFYFAr',
  'NRnMgAr',
  'NTh',
  'OB',
  'ORnFAr'],
 'Mg': ['BF', 'TiMg'],
 'N': ['CRnFAr', 'HSi'],
 'O': ['CRnFYFAr', 'CRnMgAr', 'HP', 'NRnFAr', 'OTi'],
 'P': ['CaP', 'PTi', 'SiRnFAr'],
 'Si': ['CaSi'],
 'Th': ['ThCa'],
 'Ti': ['BP', 'TiTi'],
 'e': ['HF', 'NAl', 'OMg']}

In [3]:
test_ops = {"H": ["HO", "OH"], "O": ["HH"], "e": ["H", "O"]}
test_cmp = "HOH"

### Part 1

In [4]:
def find_all_results( ops, compound ):
    results = set()
    pos = 0
    while pos < len(compound):
        if pos+1 < len(compound) and compound[pos+1] == compound[pos+1].lower():
            # two-letter element
            el_len = 2
        else:
            # one-letter element
            el_len = 1

        # read element abbreviation
        el = compound[pos:pos+el_len]

        if el in ops.keys():
            # scan through all possible reaction outputs
            for rxn in ops[el]:
                result = compound[:pos] + rxn + compound[pos+el_len:]
                results.add( result )
        
        pos += el_len
    return results


In [5]:
find_all_results( test_ops, test_cmp )

{'HHHH', 'HOHO', 'HOOH', 'OHOH'}

In [6]:
part1 = find_all_results( ops, compound )
print(len(list(part1)))

518


### Part 2

In [7]:
rev = {}
for k,v in ops.items():
    for rxn in v:
        if rxn in rev.keys():
            rev[rxn].add(k)
        else:
            rev[rxn] = set([k])

rev

{'ThF': {'Al'},
 'ThRnFAr': {'Al'},
 'BCa': {'B'},
 'TiB': {'B'},
 'TiRnFAr': {'B'},
 'CaCa': {'Ca'},
 'PB': {'Ca'},
 'PRnFAr': {'Ca'},
 'SiRnFYFAr': {'Ca'},
 'SiRnMgAr': {'Ca'},
 'SiTh': {'Ca'},
 'CaF': {'F'},
 'PMg': {'F'},
 'SiAl': {'F'},
 'CRnAlAr': {'H'},
 'CRnFYFYFAr': {'H'},
 'CRnFYMgAr': {'H'},
 'CRnMgYFAr': {'H'},
 'HCa': {'H'},
 'NRnFYFAr': {'H'},
 'NRnMgAr': {'H'},
 'NTh': {'H'},
 'OB': {'H'},
 'ORnFAr': {'H'},
 'BF': {'Mg'},
 'TiMg': {'Mg'},
 'CRnFAr': {'N'},
 'HSi': {'N'},
 'CRnFYFAr': {'O'},
 'CRnMgAr': {'O'},
 'HP': {'O'},
 'NRnFAr': {'O'},
 'OTi': {'O'},
 'CaP': {'P'},
 'PTi': {'P'},
 'SiRnFAr': {'P'},
 'CaSi': {'Si'},
 'ThCa': {'Th'},
 'BP': {'Ti'},
 'TiTi': {'Ti'},
 'HF': {'e'},
 'NAl': {'e'},
 'OMg': {'e'}}

In [8]:
max_len = 0
for k,v in rev.items():
    max_len = max( [max_len, len(list(v))] )
print(max_len)

# seems each result is generated by a unique item, so let's simplify
for k in rev.keys():
    rev[k] = list(k)[0]

1


First Attempt: Recursion -- Takes too long

In [9]:
# First attempt takes too long
def min_steps( ops, compound, max_print_depth=4, iter=[], depth = 1):
    #print("Depth:", depth)
    # Cheatery
    global best_memo
    if depth == 1:#len(iter) == 0:
        best_memo = {}

    # Base Case
    if compound in ops.keys() and ops[compound] == 'e':
        return 1

    # Memo Case
    if compound in best_memo.keys():
        return best_memo[compound]

    # Do some Iterating!
    else:
        best = None
        max_key_len = max( [len(x) for x in ops.keys()] )

        #if len(iter) <= max_print_depth:
        #    iter.append(0)

        for pos in range(len(compound)):
            #if len(iter) <= max_print_depth:
            #    iter[-1] = pos
            #    print("At pos: ", "-".join([str(x) for x in iter]))
                
            # key must start with capital
            if compound[pos] == compound[pos].lower():
                continue

            # check possible key lengths
            for i in range(max_key_len):
                # we've gone too far, quit and try next pos
                if pos + i >= len(compound):
                    break

                sub = compound[pos:pos+i+1]
                if sub in ops.keys():
                    if ops[sub] == "e":
                        continue

                    new_compound = compound[:pos] + ops[sub] + compound[pos+i+1:]
                    steps = min_steps( ops, new_compound, max_print_depth=max_print_depth, iter=iter, depth=depth+1 )
                    if steps is not None:
                        # found a path, see if it's better
                        if best is None:
                            best = steps + 1
                        else:
                            best = min( [steps+1, best] )
            # move to next character in chain

        return best   

In [10]:
test_rev = {"HH": "O", "HO": "H", "OH": "H", "H":"e", "O":"e"}
print("test 1:", min_steps(test_rev, "HOH"))
print("test 2:", min_steps(test_rev, "HOHOHO"))

test 1: 3
test 2: 6


In [11]:
print(len(compound))
min_steps(rev, compound, max_print_depth=100)

488


Second Attempt: Build from beginning -- too much memory
(probably also takes too long :D)

In [32]:
def find_min( ops, goal_compound, max_iter=9999 ):
    longest = 1
    solved = set(ops['e'])
    while True:
        print(longest+1)
        new_solved = set()
        
        # iterate all the longest chain reactions
        for cmp in solved:
            pos = 0
            # for each of those chain rxns, iterate over the molecule and see its possibilities
            while pos < len(cmp):
                # get next element in compound
                if pos+1 < len(cmp) and cmp[pos+1] == cmp[pos+1].lower():
                    el_len = 2
                else:
                    el_len = 1
                el = cmp[pos:pos+el_len]

                if el in ops.keys():
                    # add those next step chain rxns to our list
                    for rxn in ops[el]:
                        new_compound = cmp[:pos] + rxn + cmp[pos+el_len:]
                        if new_compound == goal_compound:
                            return longest + 1
                        new_solved.add( new_compound )

                pos = pos + el_len
        solved = new_solved
        longest += 1

        if longest >= max_iter:
            break

    print("did not solve :(")
    return longest, solved

In [33]:
find_min(test_ops, "HOH")

2
3


3

In [34]:
find_min(test_ops, "HOHOHO")

2
3
4
5
6


6

In [39]:
find_min(ops, compound, max_iter = 20)

2
3
4
5
6
7
8
9
10


: 

: 

Third Attempt:
* CRn only occurs at beginning of compound
* Nothing can transform to RnX
* Rn always appears second on the rxn output
* Rn does not transform
* Reactions always add go from 1 element to 2 elements, except those that insert Rn

Therefore:
1. Once you have Rn...Rn, the stuff between the Rn's is independent of the rest of the compound
2. And the last transformation to achieve WRnXZ will be ?Z -> WRnXZ,
where W is the last element before the Rn, ? => WRnX, and Z is some unknown chain of elements

Also C only appears as first element in the compound. We should probably work left-to-right on the sub-chains

## Notes

1. Seed solution with one of the front reactions CRn*
2. target is Rn(^Rn*)Rn
3. tlen is len(target)

In [12]:
import re
def listify_compound( cmp ):
    return re.findall( "[A-Z][a-z]?", cmp )
listify_compound( compound )


['C',
 'Rn',
 'Si',
 'Rn',
 'Ca',
 'P',
 'Ti',
 'Mg',
 'Y',
 'Ca',
 'P',
 'Ti',
 'Rn',
 'F',
 'Ar',
 'Si',
 'Th',
 'F',
 'Ar',
 'Ca',
 'Si',
 'Th',
 'Si',
 'Th',
 'P',
 'B',
 'Ca',
 'Ca',
 'Si',
 'Rn',
 'Si',
 'Rn',
 'Ti',
 'Ti',
 'Mg',
 'Ar',
 'P',
 'B',
 'Ca',
 'P',
 'Mg',
 'Y',
 'P',
 'Ti',
 'Rn',
 'F',
 'Ar',
 'F',
 'Ar',
 'Ca',
 'Si',
 'Rn',
 'B',
 'P',
 'Mg',
 'Ar',
 'P',
 'Rn',
 'Ca',
 'P',
 'Ti',
 'Rn',
 'F',
 'Ar',
 'Ca',
 'Si',
 'Th',
 'Ca',
 'Ca',
 'F',
 'Ar',
 'P',
 'B',
 'Ca',
 'Ca',
 'P',
 'Ti',
 'Ti',
 'Rn',
 'F',
 'Ar',
 'Ca',
 'Si',
 'Rn',
 'Si',
 'Al',
 'Y',
 'Si',
 'Th',
 'Rn',
 'F',
 'Ar',
 'Ar',
 'Ca',
 'Si',
 'Rn',
 'B',
 'F',
 'Ar',
 'Ca',
 'Ca',
 'Si',
 'Rn',
 'Si',
 'Th',
 'Ca',
 'Ca',
 'Ca',
 'F',
 'Y',
 'Ca',
 'P',
 'Ti',
 'B',
 'Ca',
 'Si',
 'Th',
 'Ca',
 'Si',
 'Th',
 'P',
 'Mg',
 'Ar',
 'Si',
 'Rn',
 'Ca',
 'P',
 'B',
 'F',
 'Y',
 'Ca',
 'Ca',
 'F',
 'Ar',
 'Ca',
 'Ca',
 'Ca',
 'Ca',
 'Si',
 'Th',
 'Ca',
 'Si',
 'Rn',
 'P',
 'Rn',
 'F',
 'Ar',
 'P',
 'B',


In [17]:
start_ops = []
end_ops = []
mid_ops = []

for k, v in ops.items():
    for rxn in v:
        rxn = listify_compound( rxn )
        if rxn[0] == "C":
            start_ops.append( [k, rxn] )
        elif rxn[1] == "Rn":
            end_ops.append( [k, rxn] )
        else:
            mid_ops.append( [k, rxn] )

print( start_ops )
print("----")
print(end_ops)
print("----")
print(mid_ops)

[['H', ['C', 'Rn', 'Al', 'Ar']], ['H', ['C', 'Rn', 'F', 'Y', 'F', 'Y', 'F', 'Ar']], ['H', ['C', 'Rn', 'F', 'Y', 'Mg', 'Ar']], ['H', ['C', 'Rn', 'Mg', 'Y', 'F', 'Ar']], ['N', ['C', 'Rn', 'F', 'Ar']], ['O', ['C', 'Rn', 'F', 'Y', 'F', 'Ar']], ['O', ['C', 'Rn', 'Mg', 'Ar']]]
----
[['Al', ['Th', 'Rn', 'F', 'Ar']], ['B', ['Ti', 'Rn', 'F', 'Ar']], ['Ca', ['P', 'Rn', 'F', 'Ar']], ['Ca', ['Si', 'Rn', 'F', 'Y', 'F', 'Ar']], ['Ca', ['Si', 'Rn', 'Mg', 'Ar']], ['H', ['N', 'Rn', 'F', 'Y', 'F', 'Ar']], ['H', ['N', 'Rn', 'Mg', 'Ar']], ['H', ['O', 'Rn', 'F', 'Ar']], ['O', ['N', 'Rn', 'F', 'Ar']], ['P', ['Si', 'Rn', 'F', 'Ar']]]
----
[['Al', ['Th', 'F']], ['B', ['B', 'Ca']], ['B', ['Ti', 'B']], ['Ca', ['Ca', 'Ca']], ['Ca', ['P', 'B']], ['Ca', ['Si', 'Th']], ['F', ['Ca', 'F']], ['F', ['P', 'Mg']], ['F', ['Si', 'Al']], ['H', ['H', 'Ca']], ['H', ['N', 'Th']], ['H', ['O', 'B']], ['Mg', ['B', 'F']], ['Mg', ['Ti', 'Mg']], ['N', ['H', 'Si']], ['O', ['H', 'P']], ['O', ['O', 'Ti']], ['P', ['Ca', 'P']], ['P', ['P

In [20]:
x = [1,2,3]
y = [1,1,2,3]
x == y

False

In [41]:
def mid_solve( have, want, mid_ops, return_ops = False ):
    # figure out if operations can get from have to want without adding any more Rn
    # returns None if impossible, or returns the operations to perform or just the number of ops depending on return_ops
    # For numeric case, result should be None or len(want) - len(have)
    
    # trivial base case
    if have == want:
        if return_ops:
            return []
        else:
            return 0
    # each step increases length, so this is impossible
    if len(have) == len(want):
        return None

    if have[0] != want[0]:
        # we need to change the first element, so start with that
        to_check = [op for op in mid_ops if op[0] == have[0]] # get all ops that xform the first elem of have
        for op in to_check:
            # see if we can solve after changing first element of have with the result of op
            result = mid_solve( op[1] + have[1:], want, mid_ops, return_ops )
            if result is not None:
                if return_ops:
                    return [op, *result]
                else:
                    return result + 1
        # couldn't get there after fixing first element. oh well :(
        return None

    if have[-1] != want[-1]:
        # we need to change the last element, so start with that
        to_check = [op for op in mid_ops if op[0] == have[-1]] # get all ops that xform the last elem of have
        for op in to_check:
            # see if we can solve after changing last element of have with the result of op
            result = mid_solve( have[:-1] + op[1], want, mid_ops, return_ops )
            if result is not None:
                if return_ops:
                    return [op, *result]
                else:
                    return result + 1
        # couldn't get there after fixing last element. oh well :(
        return None

    # first and last element are fine
    # we could possible create more hard rules, but let's just brute force it from here
    # we have to change at element, it could even be the first/last since some ops maintain the front element
    for i, el in enumerate(have):
        # test all substitutions of the ith element
        to_check = [op for op in mid_ops if op[0] == el]
        for op in to_check:
            result = mid_solve( have[:i] + op[1] + have[i+1:], want, mid_ops, return_ops )
            if result is not None:
                if return_ops:
                    return [op, *result]
                else:
                    return result + 1
        # no luck, try subbing the next element
    # end for

    # We tried everything! This is impossible!
    return None
    


    

In [48]:
inp = ["F", "Y", "Mg"]
out = ["P", "Ti", "Ti", "Mg", "Y", "B", "F"]
print( mid_solve( inp, ["P", "Mg", "Y", "Mg"], mid_ops, return_ops = True ) )
print( mid_solve( inp, ["F", "Y", "Ti", "Mg"], mid_ops, return_ops = True ) )
print( mid_solve(inp, out, mid_ops, return_ops = True) )
print( mid_solve(inp, out[:len(inp)], mid_ops) )

print( mid_solve( inp, ["Ca"]*8 + inp, mid_ops) )
print( mid_solve( inp, ["Mg"] + ["Ca"]*8 + inp, mid_ops) )

[['F', ['P', 'Mg']]]
[['Mg', ['Ti', 'Mg']]]
[['F', ['P', 'Mg']], ['Mg', ['B', 'F']], ['P', ['P', 'Ti']], ['P', ['P', 'Ti']]]
None
8
None


In [8]:
current = "AlAr"
goal = "SiRn"
last = "Si"
goal1 = [ key for key in ops.keys() if any([ last + "Rn" == v[:len(last)+2] for v in ops[key]]) ]


In [7]:
keys for any(["Si" + "Rn" == v[:4] for v in ops["P"]])

True

In [9]:
goal1

['Ca', 'P']

In [23]:
import re
# could switch to lists of elements, but ehhh
target = "SiRn"
end = target[-3:] if target[-3] == target[-3].upper() else target[-4:]

end_len = len(target)

possibles = [key for key in ops.keys() if any( [rxn[:end_len] == end for rxn in ops[key][:end_len]] ) ]


#end = re.match(r"(.*Rn)", target).groups(1)
print(end)
#solutions = [key in ops.keys() if ops[key]]
#start = []

SiRn


In [22]:
possibles

['Ca', 'P']

In [20]:
ops

{'Al': ['ThF', 'ThRnFAr'],
 'B': ['BCa', 'TiB', 'TiRnFAr'],
 'Ca': ['CaCa', 'PB', 'PRnFAr', 'SiRnFYFAr', 'SiRnMgAr', 'SiTh'],
 'F': ['CaF', 'PMg', 'SiAl'],
 'H': ['CRnAlAr',
  'CRnFYFYFAr',
  'CRnFYMgAr',
  'CRnMgYFAr',
  'HCa',
  'NRnFYFAr',
  'NRnMgAr',
  'NTh',
  'OB',
  'ORnFAr'],
 'Mg': ['BF', 'TiMg'],
 'N': ['CRnFAr', 'HSi'],
 'O': ['CRnFYFAr', 'CRnMgAr', 'HP', 'NRnFAr', 'OTi'],
 'P': ['CaP', 'PTi', 'SiRnFAr'],
 'Si': ['CaSi'],
 'Th': ['ThCa'],
 'Ti': ['BP', 'TiTi'],
 'e': ['HF', 'NAl', 'OMg']}